<a href="https://colab.research.google.com/github/kaoutar-lakdim/LM-Enhanced-Search/blob/main/Copie_de_Llama2llamaindexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pypdf
!pip install python-dotenv

In [17]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [18]:
!pip install sentence_transformers

In [19]:
!pip install llama-index

In [20]:
import tensorflow as tf
# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
    # Additional information about the GPU
    print(tf.config.list_physical_devices('GPU'))
else:
    print("GPU is NOT available. Using CPU.")

GPU is available
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/toefel").load_data()

In [25]:

from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [26]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [27]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [29]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [30]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [31]:
query_engine = index.as_query_engine()


In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

what is toefel
The word "TOEFL" is an acronym that stands for "Test of English as a Foreign Language."
explain more

The advertisement for the luxury car is attempting to appeal to the purchaser's desire for preservation, pride, and pleasure. The ad uses various means to convey this message, including:

1. Preservation: The ad highlights the car's V-12 engine, which is described as "turbinelike smoothness and awesome torque." This emphasizes the car's power and efficiency, which are likely to appeal to the purchaser's desire to preserve their investment and enjoy a reliable and powerful vehicle.
2. Pride: The ad uses language that evokes a sense of exclusivity and luxury, such as "class of one" and "the only V-12 powered motorcar for sale in America." This is likely to appeal to the purchaser's desire to show off their good taste and prosperity.
3. Pleasure: The ad emphasizes the car's luxurious features, such as "supple hides" and "electronic conveniences that pamper you." This is lik